In [19]:
from skimage import io
import random
import os
import albumentations as A
import cv2
import matplotlib.pyplot as plt

##### Usefull paths

In [21]:
imgs_path = os.path.join('data', 'originals','imgs_Red')
masks_path = os.path.join('data', 'originals', 'masks')
imgs_resized_path = os.path.join('data', 'originals', 'imgs_resized')
masks_resized_path = os.path.join('data', 'originals', 'masks_resized')
augmented_imgs_path = os.path.join('data','augmented', 'aug_imgs')
augmented_masks_path = os.path.join('data', 'augmented', 'aug_masks')

In [35]:
for directory in [augmented_imgs_path, augmented_masks_path]:
    for f in os.listdir(directory):
        full_path = os.path.join(directory, f)
        os.remove(full_path)

##### Resize images

In [ ]:
file_index = 0
for sat,num in [('L5',20), ('L8',48),('L9',6)]:
    for i in range(num):
        img = os.path.join(imgs_path, f'{i}_{sat}_R.png')
        mask = os.path.join(masks_path, f'{i}_{sat}_mask.png')

        if os.path.exists(img) == False or os.path.exists(mask) == False:
            continue
        
        image = cv2.imread(img)
        resized_img = image[10:266, 0:256]
        cv2.imwrite(os.path.join(imgs_resized_path, f'{file_index}.png'), resized_img)

        mask_img = cv2.imread(mask)
        resized_mask = mask_img[10:266, 0:256]
        cv2.imwrite(os.path.join(masks_resized_path, f'{file_index}.png'), resized_mask)
        file_index += 1
        

### Augmentation by cutting images of 128x128

In [36]:
new_size = 200
file_index = 0
for i in range(55):
    img = os.path.join(imgs_resized_path, f'{i}.png')
    mask = os.path.join(masks_resized_path, f'{i}.png')

    if os.path.exists(img) == False or os.path.exists(mask) == False:
        continue
    
    image = cv2.imread(img)
    mask_img = cv2.imread(mask)

    row = 0
    col = 0

    while row < (256 - new_size):
        col = 0
        while col < (256 - new_size):
            cutted_img = image[row:row+new_size, col:col+new_size]
            cutted_mask = mask_img[row:row+new_size, col:col+new_size]

            cutted_img = cv2.resize(cutted_img, (256, 256))
            cutted_mask = cv2.resize(cutted_mask, (256, 256))

            cv2.imwrite(os.path.join(augmented_imgs_path, f'{file_index}.png'), cutted_img)
            cv2.imwrite(os.path.join(augmented_masks_path, f'{file_index}.png'), cutted_mask)
            col += 5
            file_index += 1
        row += 5


        

##### Create and save augmented images

In [ ]:
images_to_generate = 75

aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2)
])

images = []
masks = []


i = 1
file_index = 301
num_images = len(os.listdir(imgs_path))
while i <= images_to_generate:
    number = random.randint(0, num_images - 1)
    try:
        image = os.path.join(imgs_path, f'{number}_L5_RGB.png')
        mask = os.path.join(masks_path, f'{number}_L5_mask.png')
        original_image = io.imread(image)
        original_mask = io.imread(mask)
        
        augmented = aug(image=original_image, mask=original_mask)
        transformed_image = augmented['image']
        transformed_mask = augmented['mask']

        new_image_path= "%s/%s.png" %(augmented_imgs_path, file_index)
        new_mask_path = "%s/%s.png" %(augmented_masks_path, file_index)
        io.imsave(new_image_path, transformed_image)
        io.imsave(new_mask_path, transformed_mask)
    except FileNotFoundError:
        print(f'File not found: {number}')
        continue
    i =i+1
    file_index += 1

**This code has been obtained from [this video](https://www.youtube.com/watch?v=J_XSd_u_Yew&t=359s)